In [7]:
using LoopVectorization, BenchmarkTools


For simple loops like a dot product, LoopVectorization.jl's most important optimization is to handle these tails more efficiently:



In [8]:

function mydot(a, b)
          s = 0.0
          @inbounds @simd for i ∈ eachindex(a,b)
              s += a[i]*b[i]
          end
          s
      end

function mydotavx(a, b)
          s = 0.0
          @turbo for i ∈ eachindex(a,b)
              s += a[i]*b[i]
          end
          s
      end

a = rand(256); b = rand(256);

In [4]:

@btime mydot($a, $b)

@btime mydotavx($a, $b) # performance is similar


  23.249 ns (0 allocations: 0 bytes)
  16.429 ns (0 allocations: 0 bytes)


58.46030998684942

In [5]:

a = rand(255); b = rand(255);

@btime mydot($a, $b) # with loops shorter by 1, the remainder is now 32, and it is slow

@btime mydotavx($a, $b) # performance remains mostly unchanged.


  34.661 ns (0 allocations: 0 bytes)
  14.999 ns (0 allocations: 0 bytes)


71.31644264039625

Fancier loops can be optimized

In [9]:
function mygemm!(C, A, B)
           @inbounds @fastmath for m ∈ axes(A,1), n ∈ axes(B,2)
               Cmn = zero(eltype(C))
               for k ∈ axes(A,2)
                   Cmn += A[m,k] * B[k,n]
               end
               C[m,n] = Cmn
           end
       end

mygemm! (generic function with 1 method)

In [10]:
function mygemmavx!(C, A, B)
           @turbo for m ∈ axes(A,1), n ∈ axes(B,2)
               Cmn = zero(eltype(C))
               for k ∈ axes(A,2)
                   Cmn += A[m,k] * B[k,n]
               end
               C[m,n] = Cmn
           end
       end

mygemmavx! (generic function with 1 method)

In [11]:
M, K, N = 191, 189, 171;

In [15]:
C1 = Matrix{Float64}(undef, M, N);
C2 = similar(C1); 
C3 = similar(C1);
A = randn(M, K); 
B = randn(K, N);

In [16]:
@benchmark mygemmavx!($C1, $A, $B)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  237.173 μs …  1.550 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     240.360 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   251.788 μs ± 34.283 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▆▄▄▄▃▂▁▁▁▁▁▁▁▁▁                                             ▁
  ███████████████████▇▇▇█▇▇▇▇▇▇▇▇▇▇▆▆▇▆▇▆▆▆▆▆▆▆▅▆▅▅▅▅▅▆▄▄▅▅▄▅▅ █
  237 μs        Histogram: log(frequency) by time       389 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [17]:
@benchmark mygemm!($C2, $A, $B)

BenchmarkTools.Trial: 765 samples with 1 evaluation.
 Range (min … max):  5.891 ms …  17.351 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     6.402 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.518 ms ± 665.667 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▁▁▄█▄▂▃▂▁▅▂▁▁▁▁                                             
  ▇████████████████▅▇▅▄▃▃▄▃▄▃▃▃▃▂▂▁▃▃▁▂▁▂▂▁▂▁▂▂▂▃▁▂▁▁▂▂▁▁▁▁▁▂ ▄
  5.89 ms         Histogram: frequency by time        8.95 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [18]:
using LinearAlgebra, Test


In [19]:
 @test all(C1 .≈ C2)

Test Passed
  Expression: all(C1 .≈ C2)

In [21]:
BLAS.set_num_threads(1); BLAS.get_config()

LinearAlgebra.BLAS.LBTConfig
Libraries: 
└ [ILP64] libopenblas64_.dylib

In [22]:
@benchmark mul!($C3, $A, $B)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  298.538 μs … 737.886 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     306.343 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   327.389 μs ±  43.791 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▅▅▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁                                 ▁
  █████████████████████████████████████▇▇▇▇▇▇▇▇▇▆▆▆▆▅▆▆▆▅▆▆▄▅▅▆ █
  299 μs        Histogram: log(frequency) by time        495 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [23]:
@test all(C1 .≈ C3)

Test Passed
  Expression: all(C1 .≈ C3)